# Пахомов Максим КА-12 КП-4 
# Вар-4 Arabic Handwritten Characters Dataset, kaggle.com

## Training the basic model with one convolutional layer and tuning hyperparameters:

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
import datetime

# Importing data
train_images = pd.read_csv('archive/csvTrainImages 13440x1024.csv', header=None)
train_labels = pd.read_csv('archive/csvTrainLabel 13440x1.csv', header=None)
test_images = pd.read_csv('archive/csvTestImages 3360x1024.csv', header=None)
test_labels = pd.read_csv('archive/csvTestLabel 3360x1.csv', header=None)

# Data preparation
train_images = train_images.values.reshape(-1, 32, 32, 1).astype('float32')
test_images = test_images.values.reshape(-1, 32, 32, 1).astype('float32')

train_labels -= 1
test_labels -= 1


num_classes = 28 
train_labels = to_categorical(train_labels, num_classes=num_classes)
test_labels = to_categorical(test_labels, num_classes=num_classes)

# Splitting data into train/test
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Function for building and training the model
def build_and_train_model(conv_layers=1, filters=32, kernel_size=(3, 3), padding='valid', strides=(1, 1),
                          use_batch_norm=False, dropout_rate=0.0):
    model = Sequential()

    
    for _ in range(conv_layers):
        model.add(Conv2D(filters, kernel_size=kernel_size, padding=padding, strides=strides, activation='relu', input_shape=(32, 32, 1)))
        if use_batch_norm:
            model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    if dropout_rate > 0:
        model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # TensorBoard log
    log_dir = "logs/fit/" + "conv_layers" + str(conv_layers) + "use_batch_norm=" + str(use_batch_norm) + "dropout_rate" + str(dropout_rate) + "padding" + str(padding) + "strides" + str(strides)
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Training a base model with 1 convolutional layer
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64, callbacks=[tensorboard_callback])

    return model, history

# Experiment 1: Different padding and strides
params = [
    {'padding': 'valid', 'strides': (1, 1)},
    {'padding': 'same', 'strides': (1, 1)},
    {'padding': 'same', 'strides': (2, 2)},
]

for param in params:
    print(f"Experiment with padding={param['padding']} and strides={param['strides']}")
    model, history = build_and_train_model(padding=param['padding'], strides=param['strides'])
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f'Validation accuracy: {val_acc:.4f}\n')

# Experiment 2: Alternative structures with different numbers of convolutional layers, normalisation and dropout
architectures = [
    {'conv_layers': 2, 'use_batch_norm': False, 'dropout_rate': 0.0},
    {'conv_layers': 3, 'use_batch_norm': True, 'dropout_rate': 0.2},
    {'conv_layers': 3, 'use_batch_norm': True, 'dropout_rate': 0.5},
]

for arch in architectures:
    print(f"Experiment with conv_layers={arch['conv_layers']}, use_batch_norm={arch['use_batch_norm']}, dropout_rate={arch['dropout_rate']}")
    model, history = build_and_train_model(conv_layers=arch['conv_layers'], use_batch_norm=arch['use_batch_norm'], dropout_rate=arch['dropout_rate'])
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f'Validation accuracy: {val_acc:.4f}\n')



Experiment with padding=valid and strides=(1, 1)
Epoch 1/10


2025-03-17 11:41:04.696991: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


168/168 [==============================] - 2s 12ms/step - loss: 4.5071 - accuracy: 0.0935 - val_loss: 3.0112 - val_accuracy: 0.1291
Epoch 2/10
168/168 [==============================] - 2s 12ms/step - loss: 2.8415 - accuracy: 0.1689 - val_loss: 2.6677 - val_accuracy: 0.2273
Epoch 3/10
168/168 [==============================] - 2s 12ms/step - loss: 2.4707 - accuracy: 0.2546 - val_loss: 2.3379 - val_accuracy: 0.2734
Epoch 4/10
168/168 [==============================] - 2s 12ms/step - loss: 2.0396 - accuracy: 0.3426 - val_loss: 1.9290 - val_accuracy: 0.4137
Epoch 5/10
168/168 [==============================] - 2s 13ms/step - loss: 1.5055 - accuracy: 0.5042 - val_loss: 1.4171 - val_accuracy: 0.5301
Epoch 6/10
168/168 [==============================] - 2s 12ms/step - loss: 1.0008 - accuracy: 0.6605 - val_loss: 1.1574 - val_accuracy: 0.6369
Epoch 7/10
168/168 [==============================] - 2s 13ms/step - loss: 0.6881 - accuracy: 0.7621 - val_loss: 1.0120 - val_accuracy: 0.6879
Epoch 8/10

We can see that the best result in terms of metrics is the one with padding=same, strides=(1,1). This model has an accuracy of roughly 78%.
Also after analysis of parameters and convolutional numbers analysis we can see that best parameters are 3 conv layers, 'use_batch_norm': True, 'dropout_rate': 0.2

Let's train such model and check the results:

In [2]:
!rm -rf ./logs/

In [3]:
best_model = build_and_train_model(conv_layers=3, use_batch_norm=True, dropout_rate=0.2, padding='same', strides=(1,1))[0]

# Evalutating
test_loss, test_acc = best_model.evaluate(test_images, test_labels, verbose=0)
print(f'Test accuracy: {test_acc:.4f}')

# Getting metrics
y_pred = best_model.predict(test_images)
y_true = np.argmax(test_labels, axis=1)
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_true, y_pred_classes, digits=4))
print(f"AUC: {roc_auc_score(test_labels, y_pred, multi_class='ovr'):.4f}")

Epoch 1/10
168/168 [==============================] - 5s 26ms/step - loss: 1.9304 - accuracy: 0.4243 - val_loss: 0.8334 - val_accuracy: 0.7344
Epoch 2/10
168/168 [==============================] - 4s 26ms/step - loss: 0.6538 - accuracy: 0.7833 - val_loss: 0.4360 - val_accuracy: 0.8519
Epoch 3/10
168/168 [==============================] - 5s 27ms/step - loss: 0.3972 - accuracy: 0.8670 - val_loss: 0.3370 - val_accuracy: 0.8862
Epoch 4/10
168/168 [==============================] - 5s 27ms/step - loss: 0.2857 - accuracy: 0.9062 - val_loss: 0.2874 - val_accuracy: 0.9033
Epoch 5/10
168/168 [==============================] - 4s 26ms/step - loss: 0.2081 - accuracy: 0.9289 - val_loss: 0.2843 - val_accuracy: 0.9100
Epoch 6/10
168/168 [==============================] - 4s 26ms/step - loss: 0.1682 - accuracy: 0.9430 - val_loss: 0.2583 - val_accuracy: 0.9152
Epoch 7/10
168/168 [==============================] - 5s 28ms/step - loss: 0.1365 - accuracy: 0.9526 - val_loss: 0.2550 - val_accuracy: 0.9204

For all 28 classes we've got great metrics - good result

In [4]:
%load_ext tensorboard

### Visualisation of training in TensorBoard

In [5]:
%tensorboard --logdir logs/fit

Launching TensorBoard...

## Now let's train the model on 

In [6]:
def load():
    data = np.load("../data.npz")
    X_train = data['X_train']
    y_train = data['y_train']
    X_val = data['X_val']
    y_val = data['y_val']
    X_test = data['X_test']
    y_test = data['y_test']
    return X_train,y_train,X_val,y_val,X_test,y_test

In [7]:
X_train,y_train,X_val,y_val,X_test,y_test=load()

In [8]:
X_train = X_train.reshape(-1, 64, 64, 1)
X_val = X_val.reshape(-1, 64, 64, 1)
X_test = X_test.reshape(-1, 64, 64, 1)

Робимо все те ж саме що і до цього, але тепер у вхідних та вихідних даних інший розмір, тому трохи змінимо функцію

In [9]:
# Функція для побудови та навчання моделі
def build_and_train_model_2(conv_layers=1, filters=32, kernel_size=(3, 3), padding='valid', strides=(1, 1),
                          use_batch_norm=False, dropout_rate=0.0):
    model = Sequential()


    for _ in range(conv_layers):
        model.add(Conv2D(filters, kernel_size=kernel_size, padding=padding, strides=strides, activation='relu', input_shape=(64, 64, 1)))
        if use_batch_norm:
            model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    if dropout_rate > 0:
        model.add(Dropout(dropout_rate))
    model.add(Dense(15, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # TensorBoard log
    log_dir = "logs/fit/" + "conv_layers" + str(conv_layers) + "use_batch_norm=" + str(use_batch_norm) + "dropout_rate" + str(dropout_rate) + "padding" + str(padding) + "strides" + str(strides)
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Навчання базової моделі з одним згортковим шаром
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64, callbacks=[tensorboard_callback])

    return model, history

# Експеримент 1: Різні параметри padding і strides
params = [
    {'padding': 'valid', 'strides': (1, 1)},
    {'padding': 'same', 'strides': (1, 1)},
    {'padding': 'same', 'strides': (2, 2)},
]

for param in params:
    print(f"Experiment with padding={param['padding']} and strides={param['strides']}")
    model, history = build_and_train_model_2(padding=param['padding'], strides=param['strides'])
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f'Validation accuracy: {val_acc:.4f}\n')

# Експеримент 2: Альтернативні архітектури з різними кількостями згорткових шарів, нормалізацією та дропаутом
architectures = [
    {'conv_layers': 2, 'use_batch_norm': False, 'dropout_rate': 0.0},
    {'conv_layers': 3, 'use_batch_norm': True, 'dropout_rate': 0.2},
    {'conv_layers': 3, 'use_batch_norm': True, 'dropout_rate': 0.5},
]

for arch in architectures:
    print(f"Experiment with conv_layers={arch['conv_layers']}, use_batch_norm={arch['use_batch_norm']}, dropout_rate={arch['dropout_rate']}")
    model, history = build_and_train_model_2(conv_layers=arch['conv_layers'], use_batch_norm=arch['use_batch_norm'], dropout_rate=arch['dropout_rate'])
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f'Validation accuracy: {val_acc:.4f}\n')



Experiment with padding=valid and strides=(1, 1)
Epoch 1/10
150/150 [==============================] - 6s 39ms/step - loss: 3.8772 - accuracy: 0.2427 - val_loss: 1.9236 - val_accuracy: 0.3913
Epoch 2/10
150/150 [==============================] - 6s 38ms/step - loss: 1.5165 - accuracy: 0.4990 - val_loss: 1.3205 - val_accuracy: 0.5804
Epoch 3/10
150/150 [==============================] - 6s 38ms/step - loss: 0.7533 - accuracy: 0.7447 - val_loss: 0.7423 - val_accuracy: 0.7875
Epoch 4/10
150/150 [==============================] - 6s 40ms/step - loss: 0.3360 - accuracy: 0.8915 - val_loss: 0.5394 - val_accuracy: 0.8479
Epoch 5/10
150/150 [==============================] - 6s 38ms/step - loss: 0.1516 - accuracy: 0.9536 - val_loss: 0.5573 - val_accuracy: 0.8654
Epoch 6/10
150/150 [==============================] - 6s 38ms/step - loss: 0.0775 - accuracy: 0.9797 - val_loss: 0.5381 - val_accuracy: 0.8754
Epoch 7/10
150/150 [==============================] - 6s 38ms/step - loss: 0.0467 - accuracy:

Бачимо, що параметри згортки не сильно впливають на якість моделі, в цілому всі підходять. Використаємо базові padding='valid', strides=(1, 1). Після єкспериментів з моделямі найкращим варіантом виявився при трьох згорткових шарах, use_batch_norm=True, dropout_rate=0.2.

Натренуємо таку модель і подивимося метрики на тестовому сеті

In [10]:
!rm -rf ./logs/

In [11]:
best_model = build_and_train_model_2(conv_layers=3, use_batch_norm=True, dropout_rate=0.2, padding='valid', strides=(1,1))[0]

# Оцінка на тестовій множині
test_loss, test_acc = best_model.evaluate(X_test, y_test, verbose=0)
print(f'Test accuracy: {test_acc:.4f}')

# Отримання метрик на тестовій множині
y_pred = best_model.predict(X_test)
y_true = np.argmax(y_test, axis=1)
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_true, y_pred_classes, digits=4))
print(f"AUC: {roc_auc_score(y_test, y_pred, multi_class='ovr'):.4f}")


Epoch 1/10
150/150 [==============================] - 14s 90ms/step - loss: 1.1226 - accuracy: 0.6507 - val_loss: 0.4329 - val_accuracy: 0.8608
Epoch 2/10
150/150 [==============================] - 11s 73ms/step - loss: 0.2532 - accuracy: 0.9169 - val_loss: 0.1852 - val_accuracy: 0.9488
Epoch 3/10
150/150 [==============================] - 11s 73ms/step - loss: 0.1190 - accuracy: 0.9640 - val_loss: 0.1160 - val_accuracy: 0.9708
Epoch 4/10
150/150 [==============================] - 11s 72ms/step - loss: 0.0699 - accuracy: 0.9785 - val_loss: 0.1558 - val_accuracy: 0.9513
Epoch 5/10
150/150 [==============================] - 11s 72ms/step - loss: 0.0479 - accuracy: 0.9860 - val_loss: 0.0748 - val_accuracy: 0.9825
Epoch 6/10
150/150 [==============================] - 11s 71ms/step - loss: 0.0358 - accuracy: 0.9897 - val_loss: 0.1015 - val_accuracy: 0.9754
Epoch 7/10
150/150 [==============================] - 11s 71ms/step - loss: 0.0300 - accuracy: 0.9902 - val_loss: 0.0840 - val_accuracy:

In [12]:
%tensorboard --logdir logs/fit

Launching TensorBoard...

Отримали дуже високі метрики на тестовій множині - практично ідеальна модель. Якщо порiвнювати побудовану згорткову модель та багатошаровий персептрон в попередньому КП, такий варіант буде набагато кращий по всім параметрам. Це і логічно, без згорткових шарів точність класифікації навряд чи можна довести до якихось високих значень

Висновок - отримали високу якість класифікації з моделями зі згортковими шарами. В порівнянні з моделями без згорткових шарів, вони є набагато ефективнішими, тому в класифікації будь-яких зображень спроба використати згорткові шари - гарна ідея.